# Match BGBM Collectors to Wikidata Items Using *Cosine Similarity*, Just Name Comparison

See also [match_names_BGBM-dwcagent-parsed-eventDate_vs_WikiData_cosine-similarity.ipynb](./match_names_BGBM-dwcagent-parsed-eventDate_vs_WikiData_cosine-similarity.ipynb) that includes the `eventDate` of sampling to get a life time reference for deciding name matches to WikiData.


Basically we …
- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

For the output of DarwinCore agent attribution, reconsidere `displayOrder` that it represents rather the data quality first and foremost, *not* the very name match.

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Construct data using Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb)

Out of the Wikidata items data set we create a data frame with unique canonical name strings and their counts.

In [1]:
import pandas as pd
import pprint, time, os

explain_and_show_the_data = True
this_timestamp_for_data=20240312
# this_timestamp_for_data=time.strftime('%Y%m%d')

wikidata = pd.read_csv(
    # "data/wikidata_persons_botanists_20231030_1539.csv", # inverse match: [particle +] family, given
    # "data/wikidata_persons_botanists_20231116.csv",        # match: given [+ particle] + family[+ , suffix]
    "data/wikidata_persons_botanists_20240312.csv", # with itemLabel + altLabel wyb, wye removed
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)

if explain_and_show_the_data:
    pprint.pprint(wikidata.columns)
    display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wikidata_link', 'orcid_link',
       'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='object')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q100142069,Frida Eggens,NaN,NaN,Eggens,Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
1,http://www.wikidata.org/entity/Q100142069,Frida Eggens,Frida,F.,F. Eggens,Frida Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
2,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Elizabeth,E.,E. Harrison,Elizabeth Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
3,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,NaN,NaN,Mrs A. H.,Mrs A. H.,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
4,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Mrs Arnold,M. A.,M. A. Harrison,Mrs Arnold Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

if explain_and_show_the_data:
    display(wd_matchtest)
    display(wd_matchtest_fullnames)


,canonical_string,item
,,count
0,"""The grandmother of female scientists in Ghana""",1
1,'Jan',1
2,'t. Hart,1
3,'t. Mannetje,1
4,('W.') S. W. Wong,1
...,...,...
159010,牧野富太郎(Tomitarô Makino),1
159011,百瀬静男(Sizuo Momose),1
159012,胡哲明,1


,canonical_string_fullname,item
,,count
0,"""The grandmother of female scientists in Ghana""",1
1,'Jan',1
2,'t Hart,1
3,'t Mannetje,1
4,('Wilson') Sze Wing Wong,1
...,...,...
184627,牧野富太郎(Tomitarô Makino),1
184628,百瀬静男(Sizuo Momose),1
184629,胡哲明,1


## Load Collectors Data Set

**Data sources:**

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

Technical notes:

- the corresponding objects, variable names of Nils’ python code were:
```
refactor df_avh = → = collectors
refactor df_avh['label'] = → = collectors['canonical_string_collector_parsed']
…
```

In [3]:
# unique names parsed already by ruby gem package: dwcagent

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_occurrenceIDs_20230524_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.
collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)

if explain_and_show_the_data: display(collectors)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample
3,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano,E.",parsed:E. A. Cano,cleaned:E. A. Cano,1,https://herbarium.bgbm.org/object/B100699397
39972,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murashige & Aaiki,parsed:Murashige<SEP>Aaiki,cleaned:Murashige<SEP>Aaiki,1,https://herbarium.bgbm.org/object/B101149305
7,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn,A.",parsed:A. Aaronsohn,cleaned:A. Aaronsohn,3,https://je.jacq.org/JE00010154
27181,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M.Ait Lafki...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,3,https://herbarium.bgbm.org/object/B100217620
27185,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A., Ait Lafkih,M. & Griffith...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Lafk...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,2,https://herbarium.bgbm.org/object/B100326682
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16202,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Tamanyan,...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,4,https://herbarium.bgbm.org/object/B100559866
16162,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,1,https://herbarium.bgbm.org/object/B100575827
16154,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",parsed:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файвуш...,cleaned:G.Ֆայվուշի Fayvush<SEP>Գ.<SEP>Г. Файву...,7,https://herbarium.bgbm.org/object/B100810598
17787,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich,parsed:Gabrielian Գաբրիելյան<SEP>Габриелян<SEP...,cleaned:Gabrielian Գաբրիելյան<SEP>Габриелян<SE...,1,https://herbarium.bgbm.org/object/B100088191


### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
if explain_and_show_the_data: 
    criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
    print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
    display(collectors[criterion_fullnames])

Show collecors given name has (propably) a full name (1376 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample
2328,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Bally,P.R.O., Abdallah, Raffael & Reichstein, T.",parsed:P.R.O. Bally<SEP>Raffael Abdallah<SEP>T...,cleaned:P.R.O. Bally<SEP>Raffael Abdallah<SEP>...,1,https://herbarium.bgbm.org/object/B200125981
27272,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,Kadir Bin Abdul,parsed:Kadir Bin Abdul,cleaned:Kadir Bin Abdul,1,https://herbarium.bgbm.org/object/B100184021
21225,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,Guilherme de Abreu (no. 103),parsed:Guilherme de Abreu,cleaned:Guilherme de Abreu,1,http://id.snsb.info/snsb/collection/22086/3086...
18644,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"García Adá, Luceño, Rico,E., Romero,T. & Varga...",parsed:García Adá<SEP>Rico Luceño<SEP>E. Romer...,cleaned:García Adá<SEP>Rico Luceño<SEP>E. Rome...,1,https://herbarium.bgbm.org/object/B100296455
417,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers,C. & al.",parsed:Schiers Ahagen<SEP>C.<SEP>Al,cleaned:Schiers Ahagen<SEP>C.<SEP>Al,1,https://herbarium.bgbm.org/object/B100194646
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38926,Мирзоева,Mirzoeva,NaN,NaN,NaN,NaN,NaN,NaN,Mirzoeva [Мирзоева] & Gambarian & Pogosian,parsed:Mirzoeva Мирзоева<SEP>Gambarian<SEP>Pog...,cleaned:Mirzoeva Мирзоева<SEP>Gambarian<SEP>Po...,1,https://herbarium.bgbm.org/object/B100355321
43846,Попов,Popov,NaN,NaN,NaN,NaN,NaN,NaN,Popov [Попов],parsed:Popov Попов,cleaned:Popov Попов,3,https://dr.jacq.org/DR049416
43847,Попов,Popov,NaN,NaN,NaN,NaN,NaN,NaN,Popov [Попов] & Vvedensky,parsed:Popov Попов<SEP>Vvedensky,cleaned:Popov Попов<SEP>Vvedensky,4,https://herbarium.bgbm.org/object/B101149435
17787,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich,parsed:Gabrielian Գաբրիելյան<SEP>Габриелян<SEP...,cleaned:Gabrielian Գաբրիելյան<SEP>Габриелян<SE...,1,https://herbarium.bgbm.org/object/B100088191


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
if explain_and_show_the_data:
    for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
        test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
        print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
        display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))    


----------------------------------------
show names with **particle** found 742 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
21225,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN
365,Aghababyan,M.,NaN,von,NaN,NaN,NaN,NaN
4212,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
61135,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
16932,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 15 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17455,August,Friedrich,II.,NaN,NaN,NaN,NaN,NaN
59171,Dogma,I.J.,Jr.,NaN,NaN,NaN,NaN,NaN
17185,Forsyth,W.,jr.,NaN,NaN,NaN,NaN,NaN
808,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN
26390,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17287,Sennen,NaN,NaN,NaN,NaN,NaN,Fr,NaN


Compile `canonical_string…` for the collector data we will later match the WikiData names with:

In [6]:
if explain_and_show_the_data: print("combine parts of names similar to WikiData's given name labels")
collectors['canonical_string_collector_parsed'] = collectors[['given', 'particle', 'family', 'suffix']]\
    .fillna('')\
    .apply(
        lambda this_df: "{given}{particle}{family}{suffix}".format(
            given=this_df["given"],
            particle=" " + this_df["particle"] if this_df["particle"] else '', 
            family=" " + this_df["family"] if this_df["family"] else '', 
            suffix=", " + this_df["suffix"] if this_df["suffix"] else ''
        ).strip(), axis="columns"
    )

criterion = collectors["particle"].str.contains("\w+ \w+", na=False)

# display(collectors['canonical_string_collector_parsed'][criterion].head())
if explain_and_show_the_data: 
    display(collectors[['canonical_string_collector_parsed', 'particle']][criterion].drop_duplicates().head(10))


combine parts of names similar to WikiData's given name labels


,canonical_string_collector_parsed,particle
4212,M.L. Reyna de Aguilar,Reyna de
8,H. Abbas al Ani,Abbas al
49324,J. Francisco del Aquila,Francisco del
34600,A.L.M. Marcailhou d' Aymeric,Marcailhou d'
47421,P.R.O. Ritchie in Bally,Ritchie in
42379,H. Perrier de la Bathie,Perrier de la
41744,A.M.F. Palisot de Beauvois,Palisot de
41745,A.M.F.J. Palisot de Beauvois,Palisot de
60691,D. Van der Ben,Van der
41756,F.A. Marschall von Bieberstein,Marschall von


In [7]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

if explain_and_show_the_data: display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
16202,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Tamanyan,..."
16162,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian..."
16154,Файвуш,Г.,NaN,NaN,NaN,NaN,NaN,NaN,Г. Файвуш,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian..."
17787,Գաբրիելյան,Gabrielian,NaN,NaN,NaN,NaN,NaN,NaN,Gabrielian Գաբրիելյան,Gabrielian [Գաբրիելյան; Габриелян] & Dittrich
59137,Թամանյան,Tamanian,NaN,NaN,NaN,NaN,NaN,NaN,Tamanian Թամանյան,Tamanian [Թամանյան; Таманян]


In [8]:
if explain_and_show_the_data: print("group and aggregate data to have unique name rows only for the matching of names later on")
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]) # custom function, to get the first entry
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)
collectors_unique.drop_duplicates(inplace=True)

if explain_and_show_the_data: display(collectors_unique)

group and aggregate data to have unique name rows only for the matching of names later on


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample
0,Silveira,A. A. da,NaN,NaN,NaN,NaN,NaN,NaN,A. A. da Silveira,"Silveira,A.A. da",6,https://herbarium.bgbm.org/object/B200129406a
1,Thouars,A. A. du,NaN,NaN,NaN,NaN,NaN,NaN,A. A. du Thouars,"Thouars,A. A.du Petit-",34,https://herbarium.bgbm.org/object/BW20158010
2,Bunge,A. A. von,NaN,NaN,NaN,NaN,NaN,NaN,A. A. von Bunge,"Bunge, A.A. von (no. )",1,http://id.snsb.info/snsb/collection/511901/634...
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",3,https://je.jacq.org/JE00010154
4,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Abaouz,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",5,https://herbarium.bgbm.org/object/B100217620
...,...,...,...,...,...,...,...,...,...,...,...,...
20947,Оганесиан,М.,NaN,NaN,NaN,NaN,NaN,NaN,М. Оганесиан,"Oganesian,M. [Ոգանեսիան,Մ.; Оганесиан,М.], Ter...",25,https://herbarium.bgbm.org/object/B100275643
20948,Androsov,Н.,NaN,NaN,NaN,NaN,NaN,NaN,Н. Androsov,"Androsov, Н. (no. 2404a)",1,http://id.snsb.info/snsb/collection/455833/556...
20949,Габриелян,Н.,NaN,NaN,NaN,NaN,NaN,NaN,Н. Габриелян,"Gabrieljan,N. [Գաբրիելյան,Ն.; Габриелян,Н.] & ...",1,https://herbarium.bgbm.org/object/B100190831
20950,Таманян,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Таманян,Tamanian [Թամանյան; Таманян],2,https://herbarium.bgbm.org/object/B100093359


In [9]:
# TODO continue 2023-08-21 10:28:54
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

## Set Up the Cosine Similarity and Text Search

See 
- for the application code https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb
- for reading on the topic: Taylor, Josh. 2019. ‘Fuzzy Matching at Scale’. Towards Data Science (blog). 2 July 2019. https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536.

The `ngrams`-function is used as an analyzer in the text search later.

In [10]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn # pip install sparse-dot-topn

def get_matches_df(sparse_matrix, A, B, top=100):
    """
    Get the matches of a data frame

    @param sparse_matrix: the matching matrix to apply
    @param A: the query data
    @param B: the match data
    @return: dataframe
    """
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'namematch_source_data': left_side,
                         'namematch_resource_data': right_side,
                         'namematch_similarity': similarity})

!pip install ftfy
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text

    @param string: the text string to perform the ngram splitting on
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('.', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = string.strip()
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [11]:
def calculateTFIDFmatchingOfData(query_data, match_data, cossim_ntop=1, cossim_lower_bound=0.5):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with awesome_cossim_topn() and return matched data

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param cossim_ntop: how many cossim matches each shall be calculated (default 1, i.e. the highest similarity) — increase it to get more alternative
        matches with less similarity
    @param cossim_lower_bound: where is the lower similarity cut off to regard data as similar (default 0.5)

    @requires get_get_matches_df()
    @requires ngrams()
    @requires awesome_cossim_topn()
    @requires TfidfVectorizer()

    @return: a data frame dictionary: namematch_source_data, namematch_resource_data, namematch_similarity (from @see get_matches_df())
    @rtype pd.DataFrame
    """

    import time
    time_start = time.time()

    # Vectorize Wikidata name (use fit_transform())
    print('Vectorizing data. This may take a while...')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_clean = vectorizer.fit_transform(match_data)
    # Vectorize collectors’ names (use transform())
    tf_idf_matrix_dirty = vectorizer.transform(query_data)

    duration = time.time() - time_start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    # Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
    # (lower_bound: a threshold that the element of A*B must be greater than
    #  https://github.com/ing-bank/sparse_dot_topn/blob/3f40611b0553b50c27f23c7dcffc3ca9a9e8f5b5/sparse_dot_topn/awesome_cossim_topn.py#L26C9-L26C78)
    cossim_matches = awesome_cossim_topn(
        tf_idf_matrix_dirty,
        tf_idf_matrix_clean.transpose(),
        ntop=cossim_ntop,
        lower_bound=cossim_lower_bound
    )
    print("Cossim matches calculated after %s s" % (time.time() - time_start))

    print("Get all matches together ...")
    # construct the matching data frame
    matches_df = get_matches_df(
        cossim_matches,
        query_data,
        match_data,
        top=0
    )
    print("Done. Matches calculated after %s s" % (time.time() - time_start))

    return matches_df

In [12]:
# some example data
if explain_and_show_the_data:
    print("Show ngram examples:")
    print("- simple name:", ngrams('Klazenga, N.'))
    print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
    print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
    print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[0]))
    
    # some example data
    for i, row in enumerate(range(5)):
        if (i == 0):
            print('\n(WikiData’s) canonical_string = (constructed) canonical_string_fullname:') 
        print("- {short_name} = {long_name}".format(
            short_name=wd_matchtest['canonical_string'].at[row],
            long_name=wd_matchtest_fullnames['canonical_string_fullname'].at[row]
        ))


Show ngram examples:
- simple name: ['Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N']
- data from collectors: ['A A', ' A ', 'A D', ' Du', 'Du ', 'u T', ' Th', 'Tho', 'hou', 'oua', 'uar', 'ars']
- data from match-test: ['"Th', 'The', 'he ', 'e G', ' Gr', 'Gra', 'ran', 'and', 'ndm', 'dmo', 'mot', 'oth', 'the', 'her', 'er ', 'r O', ' Of', 'Of ', 'f F', ' Fe', 'Fem', 'ema', 'mal', 'ale', 'le ', 'e S', ' Sc', 'Sci', 'cie', 'ien', 'ent', 'nti', 'tis', 'ist', 'sts', 'ts ', 's I', ' In', 'In ', 'n G', ' Gh', 'Gha', 'han', 'ana', 'na"']
- data from match-test (full name): ['"Th', 'The', 'he ', 'e G', ' Gr', 'Gra', 'ran', 'and', 'ndm', 'dmo', 'mot', 'oth', 'the', 'her', 'er ', 'r O', ' Of', 'Of ', 'f F', ' Fe', 'Fem', 'ema', 'mal', 'ale', 'le ', 'e S', ' Sc', 'Sci', 'cie', 'ien', 'ent', 'nti', 'tis', 'ist', 'sts', 'ts ', 's I', ' In', 'In ', 'n G', ' Gh', 'Gha', 'han', 'ana', 'na"']

(WikiData’s) canonical_string = (constructed) canonical_string_fullname:
- "The grandmother of female sci

## Perform the matching

In [13]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values

matches = calculateTFIDFmatchingOfData(
    collectors_names, 
    wd_matchtest['canonical_string'], 
    cossim_ntop=1 # e.g. cossim_ntop=3 would give more alternative matches as well, having lower similarities, data would increase 3 times as well
)
matches = matches.sort_values(by=['namematch_similarity'], ascending=[False])
matches = matches.reset_index(names=['old_index'])

if explain_and_show_the_data: display(matches)

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.1994729042053223 s
Cossim matches calculated after 3.7709896564483643 s
Get all matches together ...
Done. Matches calculated after 3.8351521492004395 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,9557,J. Schmidt,J. Schmidt,1.0
1,15720,R.E. Drake-Brockman,R. E. Drake-Brockman,1.0
2,7909,H. Więcław,H. Więcław,1.0
3,7910,H. Wolff,H.Wolff,1.0
4,7911,H. Wollweber,H. Wollweber,1.0
...,...,...,...,...
19109,8066,H.G. van Eck,M.H.J.van Eck-Borsboom,0.5
19110,11328,L. Catalina Matiz,Català,0.5
19111,14594,P.E. Conrick,F.E. Conradi,0.5
19112,5373,F. Catoire,F. Cati,0.5


In [14]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(
    collectors_fullnames, 
    wd_matchtest_fullnames['canonical_string_fullname'], 
    cossim_ntop=1 # 10 would give more alternative matches also with lesser similarity
)

matches_fullnames = matches_fullnames.sort_values(by=['namematch_similarity'], ascending=[False])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

if explain_and_show_the_data: display(matches_fullnames)

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.4218528270721436 s
Cossim matches calculated after 3.4834094047546387 s
Get all matches together ...
Done. Matches calculated after 3.485485315322876 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,462,Émil Bureau,Émil Bureau,1.000
1,442,Wenzel Johann Sekera,Wenzel Johann Sekera,1.000
2,232,Isa Degener,Isa Degener,1.000
3,439,Walter K. Rottensteiner,Walter K. Rottensteiner,1.000
4,230,Ignaz Grundl,Ignaz Grundl,1.000
...,...,...,...,...
458,290,Krach Staschik,Krach,0.515
459,92,Degener Hatheway,Degen,0.515
460,349,Oberst von Braun,Braun,0.510
461,200,Heinz Otto,Franz Otto Schmeil,0.507


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

Note: merging 18.770.000 collector matches earlier to wikidata was too much to calculate. Hence the descision was to make the data unique by canonical_string_collector_parsed.

In [15]:
# # join (only) abbreviated name matches with collector source data
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data', 
    how='left'
)

collectors_matches.dropna(subset=['namematch_similarity'], inplace=True)
if explain_and_show_the_data:
    print("show match results of mostly abbreviated names")
    display(collectors_matches) # 19106 rows × 16 columns

show match results of mostly abbreviated names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,Silveira,A. A. da,NaN,NaN,NaN,NaN,NaN,NaN,A. A. da Silveira,"Silveira,A.A. da",6,https://herbarium.bgbm.org/object/B200129406a,0.0,A. A. da Silveira,A. A. d. Silveira,0.772
1,Thouars,A. A. du,NaN,NaN,NaN,NaN,NaN,NaN,A. A. du Thouars,"Thouars,A. A.du Petit-",34,https://herbarium.bgbm.org/object/BW20158010,1.0,A. A. du Thouars,A. A. D. Thouars,0.719
2,Bunge,A. A. von,NaN,NaN,NaN,NaN,NaN,NaN,A. A. von Bunge,"Bunge, A.A. von (no. )",1,http://id.snsb.info/snsb/collection/511901/634...,2.0,A. A. von Bunge,A. A. v. Bunge,0.658
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",3,https://je.jacq.org/JE00010154,3.0,A. Aaronsohn,A. Aaronsohn,1.000
5,Acebey,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Acebey,"Acebey, A. (no. 563)",6,http://id.snsb.info/snsb/collection/719469/786...,4.0,A. Acebey,A. Acebey,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20937,Karjagin,İ.,NaN,NaN,NaN,NaN,NaN,NaN,İ. Karjagin,"Karjagin, İ.",1,https://herbarium.bgbm.org/object/B100601585,19109.0,İ. Karjagin,Karjagin,1.000
20938,Graf,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Graf,"Graf, Ž.",1,http://id.snsb.info/snsb/collection/480433/591...,19110.0,Ž. Graf,Graf,1.000
20939,Černeva,Ž.,NaN,NaN,NaN,NaN,NaN,NaN,Ž. Černeva,"Markova,M. & Černeva,Ž.",1,https://herbarium.bgbm.org/object/B100310826,19111.0,Ž. Černeva,Korneva,0.621
20941,Fedorova,В.,NaN,NaN,NaN,NaN,NaN,NaN,В. Fedorova,"Tikhomirov,V. [Тихомиров,В.], Fedorova,T. & St...",1,https://herbarium.bgbm.org/object/B100630485,19112.0,В. Fedorova,A. Fedorov,0.834


In [16]:
# join (only) full name matches with collector source data
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed' , right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

if explain_and_show_the_data:
    print("show match results of mostly full names")
    display(collectors_matches_fullname) # 463 rows × 16 columns

show match results of mostly full names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,Orbigny,ACVDd',NaN,NaN,NaN,NaN,NaN,NaN,ACVDd' Orbigny,"Orbigny,A.C.V.D.d'",10,https://herbarium.bgbm.org/object/B200097228,0,ACVDd' Orbigny,Alcide d' Orbigny,0.535
1,Sauze,Abbé,NaN,NaN,NaN,NaN,NaN,NaN,Abbé Sauze,"Sauze,Abbé,J.",1,https://herbarium.bgbm.org/object/B100263049,1,Abbé Sauze,Sauzé,0.572
2,Hassan,Abdisalam Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalam Sheikh Hassan,"Friis,I., Vollesen,K. & Abdisalam Sheikh Hassan",7,https://herbarium.bgbm.org/object/B100003700,2,Abdisalam Sheikh Hassan,Abdisalam S. Hassan,0.758
3,Hassan,Abdisalem Sheikh,NaN,NaN,NaN,NaN,NaN,NaN,Abdisalem Sheikh Hassan,"Friis,I, Vollesen,K., & Abdisalem Sheikh Hassan",2,https://herbarium.bgbm.org/object/B100003663,3,Abdisalem Sheikh Hassan,Abdisalam S. Hassan,0.617
4,García-Mendoza,Abisaí J.,NaN,NaN,NaN,NaN,NaN,NaN,Abisaí J. García-Mendoza,"García-Mendoza,Abisaí J. & Ramos,C.",2,https://herbarium.bgbm.org/object/B100242996,4,Abisaí J. García-Mendoza,Abisai Josue García-Mendoza,0.704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Chang,Zhao Y.,NaN,NaN,NaN,NaN,NaN,NaN,Zhao Y. Chang,"Chang, Zhao Y. (no. 2004-449)",21,http://id.snsb.info/snsb/collection/469321/575...,458,Zhao Y. Chang,Zhao Y.Chang,1.000
459,Liu,Zheng-yu,NaN,NaN,NaN,NaN,NaN,NaN,Zheng-yu Liu,"Liu,Zheng-yu",2,https://herbarium.bgbm.org/object/B100517349,459,Zheng-yu Liu,Zheng Yu Liu,1.000
460,Zhuo,Zhou,NaN,NaN,NaN,NaN,NaN,NaN,Zhou Zhuo,"Kilian,N., Raab-Straube,E. von, Zhang Jianwen,...",63,https://herbarium.bgbm.org/object/B100517202,460,Zhou Zhuo,Shou Zhou Zhang,0.717
461,Heldreich,von,NaN,NaN,NaN,NaN,NaN,NaN,von Heldreich,"Heldreich,von",12,https://dr.jacq.org/DR058755,461,von Heldreich,I. von Heldreich,0.891


In [17]:
# join all name matches together
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
if explain_and_show_the_data:
    print("show match results of all abbreviated and full names")
    display(collectors_all_matches.head())

show match results of all abbreviated and full names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,"Aaronsohn,A.",3,https://je.jacq.org/JE00010154,3.0,A. Aaronsohn,A. Aaronsohn,1.0
5399,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,E.C. Abbe,"Raup,H.M. & Abbe,E.C.",2,https://herbarium.bgbm.org/object/B100241637,4991.0,E.C. Abbe,E. C. Abbe,1.0
12797,Abbe,L.B.,NaN,NaN,NaN,NaN,NaN,NaN,L.B. Abbe,"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B.",1,https://herbarium.bgbm.org/object/BGT0003826,11721.0,L.B. Abbe,L. B. Abbe,1.0
1604,Abbot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abbot,Abbot,2,https://herbarium.bgbm.org/object/B100159967,1503.0,Abbot,Abbot,1.0
11335,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,J.R. Abbott,"Bécquer,E. & Abbott,J.R.",80,https://herbarium.bgbm.org/object/B100181131,10416.0,J.R. Abbott,J.R.Abbott,1.0


In [18]:
# Save the plain name matching results only ...
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    if not os.path.exists('data'):
        print("Make data directory for saving …")
        os.makedirs('data')

    # Set some global varialbes
    # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
    # this_timestamp_for_data=20240311

    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_%s.csv' % (
        this_timestamp_for_data
    )

    collectors_all_matches.to_csv(this_output_file)

    print("Wrote plain name matches of collector names into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

In [19]:
# old code # Join Wikidata items
# df_avh_matches_wikidata = pd.merge(df_avh_matches, df_wikidata                , left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata = pd.merge(df_avh_matches_wikidata, df_wikidata_unique, left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata.rename(columns={df_avh_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)


In [20]:
# merge now with WikiData: the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)
collectors_matches_g1_merged_wikidata.drop_duplicates(inplace=True)

In [21]:
if explain_and_show_the_data:
    print("Show example data of «Kotschy…» with Cosine Similiarity we had ~0.7 … 1.0 (Nearest Neighbour distances were from 0.0 to almost 1.0)")
    print("In old matchings without altLabel: There was a match of Kotschyi, C.G.T. → Kotschy, T.   → 0.614 → http://www.wikidata.org/wiki/Q113299  with lower similarity and a correct match to Carl Georg Theodor … :-)")
    print("In old matchings without altLabel: There was a match of Kotschy, C.G.T.  → Kotschy, C.F. → 0.824 → http://www.wikidata.org/wiki/Q86842 with higer similarity but it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/")
    print("With itemLabel + altLabel: Apart from data given with family source name only, all matches seem to be correct, but interestingly the source of “C.G. Kotschy” was matched against “Kotschy” only")
    
    criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].str.contains('Kotschy')
    # collectors.given.str.contains('^\w{3,}', na=False)
    display(collectors_matches_g1_merged_wikidata[criterion].get([
        # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        'itemLabel', 
        'canonical_string_fullname', # canonical_string_fullname contains the former itemMatchingLabel
        'wikidata_link',
    ]))

Show example data of «Kotschy…» with Cosine Similiarity we had ~0.7 … 1.0 (Nearest Neighbour distances were from 0.0 to almost 1.0)
In old matchings without altLabel: There was a match of Kotschyi, C.G.T. → Kotschy, T.   → 0.614 → http://www.wikidata.org/wiki/Q113299  with lower similarity and a correct match to Carl Georg Theodor … :-)
In old matchings without altLabel: There was a match of Kotschy, C.G.T.  → Kotschy, C.F. → 0.824 → http://www.wikidata.org/wiki/Q86842 with higer similarity but it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/
With itemLabel + altLabel: Apart from data given with family source name only, all matches seem to be correct, but interestingly the source of “C.G. Kotschy” was matched against “Kotschy” only


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,canonical_string_fullname,wikidata_link
5431,1,http://id.snsb.info/snsb/collection/22980/3175...,C.G. Kotschy,Kotschy,0.815,Theodor Kotschy,Kotschy,http://www.wikidata.org/wiki/Q113299
5432,2,https://dr.jacq.org/DR049432,Kotschy,Kotschy,1.000,Theodor Kotschy,Kotschy,http://www.wikidata.org/wiki/Q113299
5433,5,https://herbarium.bgbm.org/object/B100526350,Th Kotschy,Kotschy,0.778,Theodor Kotschy,Kotschy,http://www.wikidata.org/wiki/Q113299
5458,2494,http://id.snsb.info/snsb/collection/108230/167...,C.G.T. Kotschy,C. G. T. Kotschy,1.000,Theodor Kotschy,C. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299
5459,2494,http://id.snsb.info/snsb/collection/108230/167...,C.G.T. Kotschy,C. G. T. Kotschy,1.000,Theodor Kotschy,Carl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
5460,1,https://herbarium.bgbm.org/object/B100160086,C.G.T. Kotschyi,C. G. T. Kotschy,1.000,Theodor Kotschy,C. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299
5461,1,https://herbarium.bgbm.org/object/B100160086,C.G.T. Kotschyi,C. G. T. Kotschy,1.000,Theodor Kotschy,Carl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
15643,37,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,1.000,Theodor Kotschy,K. G. T. Kotschy,http://www.wikidata.org/wiki/Q113299
15644,37,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,1.000,Theodor Kotschy,Karl Georg Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
15645,37,http://id.snsb.info/snsb/collection/16719/2549...,K.G.T. Kotschy,K. G. T. Kotschy,1.000,Theodor Kotschy,Karl George Theodor Kotschy,http://www.wikidata.org/wiki/Q113299


In [22]:
if explain_and_show_the_data:
    print("list all column names so far, and show example data")
    pprint.pprint(collectors_matches_g1_merged_wikidata.columns)
    display(collectors_matches_g1_merged_wikidata.head())

list all column names so far, and show example data
Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod',
       'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,...,ipni,abbr,bionomia_id,yob,yod,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,Silveira,A. A. da,NaN,NaN,NaN,NaN,NaN,NaN,A. A. da Silveira,"Silveira,A.A. da",...,9650-1,Silveira,NaN,1867,1945,http://www.wikidata.org/wiki/Q19002102,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/9650-1,NaN
1,Silveira,A.A.,NaN,da,NaN,NaN,NaN,NaN,A.A. da Silveira,"da Silveira,A.A.",...,9650-1,Silveira,NaN,1867,1945,http://www.wikidata.org/wiki/Q19002102,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/9650-1,NaN
2,Thouars,A. A. du,NaN,NaN,NaN,NaN,NaN,NaN,A. A. du Thouars,"Thouars,A. A.du Petit-",...,10638-1,A.Thouars,Q2821491,1793,1864,http://www.wikidata.org/wiki/Q2821491,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/10638-1,https://bionomia.net/Q2821491
3,Bunge,A. A. von,NaN,NaN,NaN,NaN,NaN,NaN,A. A. von Bunge,"Bunge, A.A. von (no. )",...,12367-1,Bunge,Q65899,1803,1890,http://www.wikidata.org/wiki/Q65899,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12367-1,https://bionomia.net/Q65899
4,Bunge,A. A. von,NaN,NaN,NaN,NaN,NaN,NaN,A. A. von Bunge,"Bunge, A.A. von (no. )",...,12367-1,Bunge,Q65899,1803,1890,http://www.wikidata.org/wiki/Q65899,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12367-1,https://bionomia.net/Q65899


In [23]:
if explain_and_show_the_data: print("select useful columns for final data results, and for preparation of the DarwinCore attribution output …")
collectors_wikidata_cossimOrKmeans = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod'
     # , 'wyb'
    ]
].copy()

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossimOrKmeans.sort_values(
    # by=['namematch_similarity', 'family', 'given'], 
    # ascending=[False, True, True], 
    by=['canonical_string_collector_parsed', 'namematch_similarity', 'family', 'given'], 
    ascending=[True, False, True, True], 
    inplace=True
)

if explain_and_show_the_data: display(collectors_wikidata_cossimOrKmeans)

select useful columns for final data results, and for preparation of the DarwinCore attribution output …


,canonical_string_collector_parsed,family,given,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,source_data,namematch_source_data,namematch_resource_data,namematch_similarity,item,...,itemLabel,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod
0,A. A. da Silveira,Silveira,A. A. da,6,https://herbarium.bgbm.org/object/B200129406a,"Silveira,A.A. da",A. A. da Silveira,A. A. d. Silveira,0.772,http://www.wikidata.org/entity/Q19002102,...,Alvaro Astolpho da Silveira,NaN,209113140,NaN,887.0,9650-1,Silveira,NaN,1867,1945
2,A. A. du Thouars,Thouars,A. A. du,34,https://herbarium.bgbm.org/object/BW20158010,"Thouars,A. A.du Petit-",A. A. du Thouars,A. A. D. Thouars,0.719,http://www.wikidata.org/entity/Q2821491,...,Abel Aubert Dupetit Thouars,NaN,42142942,0000000080191505,69096.0,10638-1,A.Thouars,Q2821491,1793,1864
3,A. A. von Bunge,Bunge,A. A. von,1,http://id.snsb.info/snsb/collection/511901/634...,"Bunge, A.A. von (no. )",A. A. von Bunge,A. A. v. Bunge,0.658,http://www.wikidata.org/entity/Q65899,...,Alexander Bunge,NaN,73966865,0000000081539796,467.0,12367-1,Bunge,Q65899,1803,1890
4,A. A. von Bunge,Bunge,A. A. von,1,http://id.snsb.info/snsb/collection/511901/634...,"Bunge, A.A. von (no. )",A. A. von Bunge,A. A. v. Bunge,0.658,http://www.wikidata.org/entity/Q65899,...,Alexander Bunge,NaN,73966865,0000000081539796,467.0,12367-1,Bunge,Q65899,1803,1890
9,A. Aaronsohn,Aaronsohn,A.,3,https://je.jacq.org/JE00010154,"Aaronsohn,A.",A. Aaronsohn,A. Aaronsohn,1.000,http://www.wikidata.org/entity/Q2086130,...,Aaron Aaronsohn,NaN,2795076,0000000109488581,30592.0,23-1,Aarons.,Q2086130,1876,1919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15737,İ. Karjagin,Karjagin,İ.,1,https://herbarium.bgbm.org/object/B100601585,"Karjagin, İ.",İ. Karjagin,Karjagin,1.000,http://www.wikidata.org/entity/Q4216461,...,Ivan Ivanovich Karyagin,NaN,NaN,NaN,71228.0,4649-1,Karjagin,NaN,1894,1966
6137,Ž. Graf,Graf,Ž.,1,http://id.snsb.info/snsb/collection/480433/591...,"Graf, Ž.",Ž. Graf,Graf,1.000,http://www.wikidata.org/entity/Q12807671,...,Siegmund Graf,NaN,170066571,0000000118968261,10265.0,3314-1,Graf,NaN,1801,1838
23551,Ž. Černeva,Černeva,Ž.,1,https://herbarium.bgbm.org/object/B100310826,"Markova,M. & Černeva,Ž.",Ž. Černeva,Korneva,0.621,http://www.wikidata.org/entity/Q36546725,...,E.I. Korneva,NaN,NaN,NaN,81989.0,4966-1,Korneva,NaN,<NA>,<NA>
2784,В. Fedorova,Fedorova,В.,1,https://herbarium.bgbm.org/object/B100630485,"Tikhomirov,V. [Тихомиров,В.], Fedorova,T. & St...",В. Fedorova,A. Fedorov,0.834,http://www.wikidata.org/entity/Q4493902,...,Aleksandr Fyodorov,NaN,128586736,0000000049643948,80361.0,2665-1,Al.Fed.,NaN,1906,1982


In [24]:
# # TODO further evaluation or filtering, counting, clean up aso.
do_custom_data_aggregation=False
if do_custom_data_aggregation:
    if not os.path.exists('data'):
        os.makedirs('data')

    # bgbm_collectors_cosine-similarity_wikidata-botanists_%s.csv
    this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_%s.csv' % (
        this_timestamp_for_data
    )

    collectors_wikidata_cossimOrKmeans.to_csv(this_output_file)

    print("Wrote matches of collector names into %s (%d kB)" % 
        (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
    )

## Output Mapping to DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [25]:
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
collectors_wikidata_cossimOrKmeans = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
     'yob', 'yod'
     # , 'wyb'
    ]
].copy()

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossimOrKmeans.sort_values(
    by=['canonical_string_collector_parsed', 'namematch_similarity', 'family', 'given'], 
    ascending=[True, False, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossimOrKmeans.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_similarity", 
    "source_data", 
    "itemLabel", 
    "item",
    'yob', 'yod'
]).copy().drop_duplicates(ignore_index=True)

dwcagent_attr_output.drop_duplicates(inplace=True, ignore_index=True)

dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['namematch_similarity_annotation'] = dwcagent_attr_output['namematch_similarity'].astype(str).str.replace(r'(.+)', '\\1 (cos similarity)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_similarity_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

# combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
# dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")
dwcagent_attr_output.assign(life_time_periode = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?") )


# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["custom_score_lifetime_data"] = 0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_data_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & pd.notnull(dwcagent_attr_output["yod"]),
    "custom_score_lifetime_data"
] = 1
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & dwcagent_attr_output["yod"].isnull(),
    "custom_score_lifetime_data"
] = 0.5
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & pd.notnull(dwcagent_attr_output["yod"]),
    "custom_score_lifetime_data"
] = 0.5
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & dwcagent_attr_output["yod"].isnull(),
    "custom_score_lifetime_data"
] = 0


# annotations True cases
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & pd.notnull(dwcagent_attr_output["yod"]), 
    "custom_score_lifetime_data_annotation"
] = "life time known"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    pd.notnull(dwcagent_attr_output["yob"]) & dwcagent_attr_output["yod"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & pd.notnull(dwcagent_attr_output["yod"]), 
    "custom_score_lifetime_data_annotation"
] = "year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob"].isnull() & dwcagent_attr_output["yod"].isnull(), 
    "custom_score_lifetime_data_annotation"
] = "unknown life time"


dwcagent_attr_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

dwcagent_attr_output['custom_score_overall'] = (
    dwcagent_attr_output['namematch_similarity'] * \
    (
        ( dwcagent_attr_output["custom_score_lifetime_data"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)

dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (data life time score);"
                " {lifetime_score_annote} (data life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_similarity_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime_data"],
    lifetime_score_annote=row["custom_score_lifetime_data_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to overall score
dwcagent_attr_output.sort_values(
    by=['canonical_string_collector_parsed', 'custom_score_overall', 'family', 'given'], 
    ascending=[True, False, True, True], 
    # by=['family', 'given', 'canonical_string_collector_parsed', 'custom_score_overall'], 
    # ascending=[True, True, True, False], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
if explain_and_show_the_data:
    print("example data: names having year of birth (yob)")
    display(dwcagent_attr_output.loc[pd.notnull(dwcagent_attr_output['yob'])].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))
    print("example data: names missing the year of birth (yob)")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob'].isnull()].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))

example data: names having year of birth (yob)


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,custom_score_lifetime_data,custom_score_lifetime_data_annotation
0,A. A. da Silveira,Alvaro Astolpho da Silveira,0.386,0.772 (cos similarity); 0.39 (score overall); ...,0.0,0.772,,1.0,life time known
1,A. A. du Thouars,Abel Aubert Dupetit Thouars,0.360,0.719 (cos similarity); 0.36 (score overall); ...,0.0,0.719,,1.0,life time known
2,A. A. von Bunge,Alexander Bunge,0.329,0.658 (cos similarity); 0.33 (score overall); ...,0.0,0.658,,1.0,life time known
3,A. Aaronsohn,Aaron Aaronsohn,0.500,1.0 (cos similarity); 0.50 (score overall); (...,0.0,1.000,,1.0,life time known
4,A. Acebey,Amparo Acebey,0.250,1.0 (cos similarity); 0.25 (score overall); (...,0.0,1.000,,0.5,year of death is missing


example data: names missing the year of birth (yob)


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,custom_score_lifetime_data,custom_score_lifetime_data_annotation
6,A. Ackermann,Manfred Ackermann,0.000,0.914 (cos similarity); 0.00 (score overall); ...,0.0,0.914,,0.0,unknown life time
13,A. Al-Farsi,Dee Dee Al-Farishy,0.000,0.554 (cos similarity); 0.00 (score overall); ...,0.0,0.554,,0.0,unknown life time
18,A. Alfaro Gonzalez,A. Alfaro-García,0.163,0.653 (cos similarity); 0.16 (score overall); ...,0.0,0.653,,0.5,year of birth is missing
21,A. Alvarado Mendez,Marco Antonio Alvarado Vásquez,0.000,0.671 (cos similarity); 0.00 (score overall); ...,0.0,0.671,,0.0,unknown life time
30,A. Antonietti,Livio Antonielli,0.000,0.631 (cos similarity); 0.00 (score overall); ...,0.0,0.631,,0.0,unknown life time


In [26]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'namematch_similarity':               'custom_namematch_similarity'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID',  # no DwC agent standard (yet)?
        'verbatimName',  # source_data
        'alternateName', # canonical_string_collector_parsed (actual collector name)
        'displayOrder',  # shall start from 1, 2, 3 … represents the available data quality not the match in the first place
        'name', # itemLabel is interpreted as the preferred name
        'attributionRemarks',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_similarity',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime_data' # keep it for calculation convenience, no standard in DwC agent
    ]
)
if explain_and_show_the_data:
    print("show column-reduced and mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …")    
    display(dwcagent_attr_output.drop(['agentType', 'action', 'agentIdentifierType'], axis='columns').head(20))


show column-reduced and mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …


,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,identifier,custom_score_overall,custom_namematch_similarity,custom_score_multiple_names,custom_score_lifetime_data
0,https://herbarium.bgbm.org/object/B200129406a,"Silveira,A.A. da",A. A. da Silveira,1,Alvaro Astolpho da Silveira,0.772 (cos similarity); 0.39 (score overall); ...,http://www.wikidata.org/entity/Q19002102,0.386,0.772,0.0,1.0
1,https://herbarium.bgbm.org/object/BW20158010,"Thouars,A. A.du Petit-",A. A. du Thouars,1,Abel Aubert Dupetit Thouars,0.719 (cos similarity); 0.36 (score overall); ...,http://www.wikidata.org/entity/Q2821491,0.360,0.719,0.0,1.0
2,http://id.snsb.info/snsb/collection/511901/634...,"Bunge, A.A. von (no. )",A. A. von Bunge,1,Alexander Bunge,0.658 (cos similarity); 0.33 (score overall); ...,http://www.wikidata.org/entity/Q65899,0.329,0.658,0.0,1.0
3,https://je.jacq.org/JE00010154,"Aaronsohn,A.",A. Aaronsohn,1,Aaron Aaronsohn,1.0 (cos similarity); 0.50 (score overall); (...,http://www.wikidata.org/entity/Q2086130,0.500,1.000,0.0,1.0
4,http://id.snsb.info/snsb/collection/719469/786...,"Acebey, A. (no. 563)",A. Acebey,1,Amparo Acebey,1.0 (cos similarity); 0.25 (score overall); (...,http://www.wikidata.org/entity/Q5673219,0.250,1.000,0.0,0.5
5,https://herbarium.bgbm.org/object/B100393538,"Álvarez de Zayas,A., Aceres,A., Bässler,M., Bi...",A. Aceres,1,Giuseppe Acerbi,0.617 (cos similarity); 0.31 (score overall); ...,http://www.wikidata.org/entity/Q55007624,0.308,0.617,0.0,1.0
6,https://herbarium.bgbm.org/object/B100092853,"Weigend,M., Ackermann,A. & Castillo,J.A.",A. Ackermann,1,Manfred Ackermann,0.914 (cos similarity); 0.00 (score overall); ...,http://www.wikidata.org/entity/Q47112660,0.000,0.914,0.0,0.0
7,https://herbarium.bgbm.org/object/B100629441,"Zardini,E.M. & Acosta,A.",A. Acosta,1,Cristóbal Acosta,0.893 (cos similarity); 0.45 (score overall); ...,http://www.wikidata.org/entity/Q384460,0.446,0.893,0.0,1.0
8,http://id.snsb.info/snsb/collection/513255/635...,"Ade, A.",A. Ade,1,Alfred Ade,1.0 (cos similarity); 0.50 (score overall); (...,http://www.wikidata.org/entity/Q8195254,0.500,1.000,0.0,1.0
9,https://herbarium.bgbm.org/object/B101010034,"Balkwill,K., McCallum,D.A., Mbatha,H.P., Adebo...",A. Adebowale,1,Alfred Ade,0.571 (cos similarity); 0.29 (score overall); ...,http://www.wikidata.org/entity/Q8195254,0.286,0.571,0.0,1.0


In [27]:
if explain_and_show_the_data:
    # this_name = "Kotsch"
    # print("show examples of a name containing “{}” …".format(this_name))
    # criterion = dwcagent_attr_output['alternateName'].filter(regex=".*Kotsch.*")
    # criterion = dwcagent_attr_output.alternateName.str.contains("Kotschy")
    # -------
    print("show column-reduced examples of ?multiple name cases …")

    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda x: x < 0 )
    
    display(dwcagent_attr_output[criterion].drop(['agentType', 'action', 'agentIdentifierType'], axis='columns').head(20))

show column-reduced examples of ?multiple name cases …


,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,identifier,custom_score_overall,custom_namematch_similarity,custom_score_multiple_names,custom_score_lifetime_data
16,https://herbarium.bgbm.org/object/B100810598,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",A. Aleksanyan,1,Anatolij Aleksandrovič Ničiporovič,0.625 (cos similarity); 0.16 (score overall); ...,http://www.wikidata.org/entity/Q26244283,0.156,0.625,-0.5,1.0
17,https://herbarium.bgbm.org/object/B100810598,"Fayvush,G. [Ֆայվուշի,Գ.; Файвуш Г.], Oganesian...",A. Aleksanyan,2,A.A. Richter,0.625 (cos similarity); 0.16 (score overall); ...,http://www.wikidata.org/entity/Q4394909,0.156,0.625,-0.5,1.0
25,https://herbarium.bgbm.org/object/B100212980,"Andersson,A. & Franzén,R.",A. Andersson,1,Axel Andersson,1.0 (cos similarity); 0.00 (score overall); (...,http://www.wikidata.org/entity/Q123652899,0.000,1.000,-0.5,0.5
26,https://herbarium.bgbm.org/object/B100212980,"Andersson,A. & Franzén,R.",A. Andersson,2,I. Anita Andersson,1.0 (cos similarity); 0.00 (score overall); (...,http://www.wikidata.org/entity/Q21505194,0.000,1.000,-0.5,0.5
55,http://id.snsb.info/snsb/collection/463093/564...,"Barmin, A.",A. Barmin,1,Eugenius Warming,0.578 (cos similarity); 0.14 (score overall); ...,http://www.wikidata.org/entity/Q355888,0.144,0.578,-0.5,1.0
56,http://id.snsb.info/snsb/collection/463093/564...,"Barmin, A.",A. Barmin,2,Eugenius Warming,0.578 (cos similarity); 0.14 (score overall); ...,http://www.wikidata.org/entity/Q355888,0.144,0.578,-0.5,1.0
70,https://je.jacq.org/JE04009579,"Bellamy,A.",A. Bellamy,1,Charles Lawrence Bellamy,0.895 (cos similarity); 0.22 (score overall); ...,http://www.wikidata.org/entity/Q19115006,0.224,0.895,-0.5,1.0
71,https://je.jacq.org/JE04009579,"Bellamy,A.",A. Bellamy,2,R.E. Bellamy,0.895 (cos similarity); -0.22 (score overall);...,http://www.wikidata.org/entity/Q36658695,-0.224,0.895,-0.5,0.0
80,http://id.snsb.info/snsb/collection/455323/555...,"Bertoloni, A.",A. Bertoloni,1,Antonio Bertoloni Jr.,1.0 (cos similarity); 0.25 (score overall); (...,http://www.wikidata.org/entity/Q120469486,0.250,1.000,-0.5,1.0
81,http://id.snsb.info/snsb/collection/455323/555...,"Bertoloni, A.",A. Bertoloni,2,Antonio Bertoloni,1.0 (cos similarity); 0.25 (score overall); (...,http://www.wikidata.org/entity/Q599936,0.250,1.000,-0.5,1.0


In [28]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# meise_collectors_cosine-similarity_wikidata-botanists_%s.csv
this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_20240312.csv (6985 kB)


## Documentation

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
occurrenceID_collectors_count | count of all occurrenceID of one particular collector name
occurrenceID_collectors_firstsample | a data sample of an occurrenceID 
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
namematch_source_data | matched name of the collector data set
namematch_resource_data | matched name of Wikidata the collector was tried to matched to
namematch_similarity | calculated cosine-similarity
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label (perhaps similar to the full name)
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))